In [ ]:
import itertools
import pathlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from matplotlib import rc, rcParams

%matplotlib inline

%load_ext autoreload
%autoreload 2

rc('text', usetex=True)
font = {'family': 'Times New Roman', 'weight': 'bold', 'size': 14}
rc('font', **font)
rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']

In [ ]:
name_vals = []
model_vals = []
solver_vals = []
loss_vals = []
overall_scores_rows = []
columns = None

def get_solver_text(solver):
    return {'orig': 'original', 'fNMS': 'feature-NMS'}[solver]

def iter_results_files(*experiments):
    for dir_name, name in experiments:
        yield from zip(
            itertools.repeat(name),
            pathlib.Path(dir_name).rglob('eval_results.csv')
        )

for name, results_file in iter_results_files(('eval_uadt_emb_ga', r'emb.\ head'), ('eval_uadt_emb_orig_ga', r'base\ model')):
    model_dir = results_file.parent.parent
    solver_dir = model_dir.parent
    loss_dir = solver_dir.parent

    name_vals.append(name + r',\ ' + solver_dir.stem + ('.' if 'orig' in solver_dir.stem else '') + r'\ solver')
    model_vals.append(model_dir.stem)
    solver_vals.append(get_solver_text(solver_dir.stem))
    loss_vals.append(loss_dir.stem)

    df = pd.read_csv(str(results_file), index_col=0)
    columns = df.columns
    overall_scores = df.iloc[-1]
    overall_scores_rows.append(overall_scores)

df_orig = pd.DataFrame(overall_scores_rows, columns=columns)
df_orig.insert(0, 'name', name_vals)
df_orig.insert(1, 'model', model_vals)
df_orig.insert(2, 'solver', solver_vals)
df_orig.insert(3, 'loss', loss_vals)
df_orig.reset_index(drop=True, inplace=True)

In [ ]:
df_orig.rename(
        columns={
            'num_frames': 'frames',
            'num_matches': 'matches',
            'num_switches': 'switches',
            'num_false_positives': 'FP',
            'num_misses': 'FN',
            'num_objects': 'objs',
            'num_predictions': 'preds',
            'num_fragmentations': 'fragms',
            'mostly_tracked': 'MT',
            'partially_tracked': 'PT',
            'precision': 'prec',
            'recall': 'rec',
            'idf1': 'IDF1',
            'mota': 'MOTA',
            'motp': 'MOTP'
        },
        inplace=True
    )

In [ ]:
def build_model_iters_mask(df, *iters):
    mask = None
    
    for iter in iters:
        iter_str = f'{iter * 1000:07d}'
        curr_mask = df['model'] == iter_str
        mask = curr_mask if mask is None else mask | curr_mask
    
    return mask

iters_mask = build_model_iters_mask(df_orig, 15, 30, 45, 60, 75, 90, 105)
df = df_orig[iters_mask]

df.sort_values(by=['name', 'solver', 'model', 'MOTA', 'MOTP', 'prec', 'rec'], ascending=True, inplace=True)

In [ ]:
def make_row_visual_props_builder(df):
    base_colors =  list(mcolors.CSS4_COLORS.keys())
    base_colors = [
        '#000000',  # black
        '#ff0000',  # red
        '#00ff00',  # green
        '#0000ff',  # blue
        '#ff00ff',  # magenta
        '#bbbb00',  # yellow
        '#00ffff',  # cyan
        '#964b00',  # brown
        '#ffa500',  # orange
        '#30d5c8',  # turquoise
        '#bc8f8f',  # rosy brown
        '#d4a437',  # gold
        '#3cb371',  # medium sea green
    ] * 2
    unique_models = df['model'].unique().tolist()
    size = 32 ** 2

    def _get_row_visual_props(data_row):
        name = data_row['name']
        model = data_row['model']

        color = base_colors[unique_models.index(model)]
        if data_row['loss'] == 'triplet':
            marker = 's' if data_row['solver'] == 'original' else '^'
        else:
            marker = 'o'
        label = rf'${name}, {str(int(model))}$'
        text = None

        return {
            'size': size, 
            'color': color,
            'marker': marker,
            'label': label,
            'text': text
        }
    
    return _get_row_visual_props
        
def plot_tracker_2d_comparison(
    df, visual_props_getter, x_col, y_col, x_label, y_label, x_units=None, y_units=None
):
    def _build_axis_label(text, units=None):
        label = rf'$\textbf{{{text}}}$'
        if units is not None:
            label += f' [{units}]'
        return label
    
    fig, ax = plt.subplots(figsize=(14, 8), nrows=1, ncols=1)

    for _, data_row in df.iterrows():
        x, y = data_row[x_col], data_row[y_col]
        visual_props = visual_props_getter(data_row)

        size = visual_props.get('size')
        color = visual_props.get('color')
        marker = visual_props.get('marker')
        label = visual_props.get('label')
        text = visual_props.get('text')

        ax.scatter(
            x, y, s=size, c=color, label=label, marker=marker, edgecolors='black',
            linewidth=2, alpha=0.7, antialiased=True
        )

        if text is not None:
            ax.text(x, y, s=text, fontdict={'weight': 'bold', 'size': 20})

    x_label_formatted = _build_axis_label(x_label, x_units)
    y_label_formatted = _build_axis_label(y_label, y_units)
    
    ax.set_xlabel(x_label_formatted)
    ax.set_ylabel(y_label_formatted)
    ax.legend(loc='best', markerscale=0.5, fontsize=14)
    ax.spines.top.set_visible(False)
    ax.spines.right.set_visible(False)

    fig.tight_layout()

    return fig

visual_props_getter = make_row_visual_props_builder(df)
fig_mota_motp = plot_tracker_2d_comparison(
    df, visual_props_getter, 'MOTA', 'MOTP', 'MOTA', 'MOTP'
)
fig_mota_motp.savefig('tracker_cmp_plot_mota_motp.pdf', dpi=200)

In [ ]:
fig_rec_prec = plot_tracker_2d_comparison(
    df, visual_props_getter, 'rec', 'prec', 'Recall', 'Precision'
)
fig_rec_prec.savefig('tracker_cmp_plot_rec_prec.pdf', dpi=200)

In [ ]:
df.sort_values(by=['loss', 'solver', 'MOTA', 'MOTP', 'prec', 'rec'], ascending=False)